1.Create your own regression dataset (or make the one we created in "Create data to view and fit" bigger) and build fit a model to it.


In [1]:
import numpy as np
import tensorflow as tf
X = np.arange(1,100,2)
X.shape,X

((50,),
 array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
        35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67,
        69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99]))

In [2]:
y = np.array(7*X + 3)
y.shape,y.dtype,y

((50,),
 dtype('int64'),
 array([ 10,  24,  38,  52,  66,  80,  94, 108, 122, 136, 150, 164, 178,
        192, 206, 220, 234, 248, 262, 276, 290, 304, 318, 332, 346, 360,
        374, 388, 402, 416, 430, 444, 458, 472, 486, 500, 514, 528, 542,
        556, 570, 584, 598, 612, 626, 640, 654, 668, 682, 696]))

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
# df = pd.DataFrame(data=X,columns=y)
# df
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)
X_train.shape,X_test.shape


((40,), (10,))

In [4]:
# Creating the model
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100,activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer=tf.keras.optimizers.SGD(),loss=tf.keras.losses.mae,metrics=['mae'])
model.fit(X_train,y_train,epochs=5)

2023-08-30 21:28:36.687559: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 21:28:36.692704: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2994340000 Hz
2023-08-30 21:28:36.693433: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x23bb1f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-08-30 21:28:36.693457: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Epoch 1/5
2/2 [==============================] - 0s 996us/step - loss: 316.7729 - mae: 316.7729
Epoch 2/5
2/2 [==============================] - 0s 1ms/step - loss: 180.2518 - mae: 180.2518
Epoch 3/5
2/2 [==============================] - 0s 801us/step - loss: 233.3665 - mae: 233.3665
Epoch 4/5
2/2 [==============================] - 0s 730us/step - loss: 110.9875 - mae: 110.9875
Epoch 5/5
2/2 [==============================] - 0s 726us/step - loss: 149.6074 - mae: 149.6074


2.Try building a neural network with 4 Dense layers and fitting it to your own regression dataset, how does it perform?


In [5]:
tf.random.set_seed(42)
model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(100,activation='relu',input_shape=(1,)),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(1,activation='relu'),
])
model_2.compile(optimizer=tf.keras.optimizers.SGD(),loss=tf.keras.losses.mae,metrics=['mae'])
model_2.fit(X_train,y_train,epochs=5)

Epoch 1/5
2/2 [==============================] - 0s 1ms/step - loss: 329.2938 - mae: 329.2938
Epoch 2/5
2/2 [==============================] - 0s 837us/step - loss: 355.5905 - mae: 355.5905
Epoch 3/5
2/2 [==============================] - 0s 756us/step - loss: 335.8500 - mae: 335.8500
Epoch 4/5
2/2 [==============================] - 0s 802us/step - loss: 335.8500 - mae: 335.8500
Epoch 5/5
2/2 [==============================] - 0s 808us/step - loss: 335.8500 - mae: 335.8500


3.Try and improve the results we got on the insurance dataset, some things you might want to try include:
* Building a larger model (how does one with 4 dense layers go?).
*Increasing the number of units in each layer.
* Lookup the documentation of Adam and find out what the first parameter is, what happens if you increase it by 10x?
What happens if you train for longer (say 300 epochs instead of 200)?


In [6]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
# Read the csv
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [7]:
# To prepare we need to use scikit-learn
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split


# Create a column transformer
# We do not scale our target variable but we do scale our feature variable
# age bmi and children are numbers so they need to be normalised
# sex smoker and region need to be onehotencoded like previous because they are strings
ct = make_column_transformer(
    (MinMaxScaler(),['age','bmi','children']),
    (OneHotEncoder(handle_unknown='ignore'), ['sex',"smoker",'region'])
)
# Create X and y 
X = insurance.drop('charges', axis=1)
y = insurance['charges']

# Build our train and test data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Fit the Column transformer to our training data
ct.fit(X_train)

# Transform training and test data with normalization (MinMaxScaler) and OneHotEncoder
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [8]:
# Creating a model
tf.random.set_seed(42)
insurance_model = tf.keras.Sequential([
    tf.keras.layers.Dense(150,activation='relu',input_shape=(11,)),
    tf.keras.layers.Dense(150,activation='relu'),
    tf.keras.layers.Dense(150,activation='relu'),
    tf.keras.layers.Dense(1)
])
insurance_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss=tf.keras.losses.mae,metrics=['mae'])
insurance_model.fit(X_train_normal,y_train,epochs=500)

Epoch 1/500
34/34 [==============================] - 0s 1ms/step - loss: 10654.6025 - mae: 10654.6025
Epoch 2/500
34/34 [==============================] - 0s 1ms/step - loss: 5992.5200 - mae: 5992.5200
Epoch 3/500
34/34 [==============================] - 0s 1ms/step - loss: 3608.5452 - mae: 3608.5452
Epoch 4/500
34/34 [==============================] - 0s 1ms/step - loss: 3453.7356 - mae: 3453.7356
Epoch 5/500
34/34 [==============================] - 0s 1ms/step - loss: 3386.9421 - mae: 3386.9421
Epoch 6/500
34/34 [==============================] - 0s 1ms/step - loss: 3271.1462 - mae: 3271.1462
Epoch 7/500
34/34 [==============================] - 0s 1ms/step - loss: 3203.7964 - mae: 3203.7964
Epoch 8/500
34/34 [==============================] - 0s 1ms/step - loss: 3124.9893 - mae: 3124.9893
Epoch 9/500
34/34 [==============================] - 0s 1ms/step - loss: 3024.9221 - mae: 3024.9221
Epoch 10/500
34/34 [==============================] - 0s 1ms/step - loss: 2927.2930 - mae: 2927.29

4. Import the Boston pricing dataset from TensorFlow tf.keras.datasets and model it.

In [13]:
# import tf.keras.datasets.boston_housing
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path='boston_housing.npz', test_split=0.2, seed=113
)
x_train.shape, x_test.shape, y_train.shape,y_test.shape

((404, 13), (102, 13), (404,), (102,))

In [14]:
x_train

array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
        3.95380e+02, 3.11000e+00],
       [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.75520e+02, 3.26000e+00],
       ...,
       [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
        3.62250e+02, 7.83000e+00],
       [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        2.61950e+02, 1.57900e+01],
       [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
        3.76700e+02, 4.38000e+00]])

In [15]:
y_train

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

In [60]:
# Creating model
tf.random.set_seed(42)
boston = tf.keras.Sequential([
    tf.keras.layers.Dense(200,activation=None,input_shape=(13,)),
    tf.keras.layers.Dense(200,activation='relu',),
    tf.keras.layers.Dense(200,activation='relu',),
    tf.keras.layers.Dense(200,activation='relu',),
    tf.keras.layers.Dense(1)
])
boston.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss=tf.keras.losses.mae,metrics=['mae'])
boston.fit(x_train,y_train,epochs=500)

Epoch 1/500
13/13 [==============================] - 0s 2ms/step - loss: 94.2898 - mae: 94.2898
Epoch 2/500
13/13 [==============================] - 0s 2ms/step - loss: 7.3521 - mae: 7.3521
Epoch 3/500
13/13 [==============================] - 0s 2ms/step - loss: 6.0581 - mae: 6.0581
Epoch 4/500
13/13 [==============================] - 0s 2ms/step - loss: 5.4290 - mae: 5.4290
Epoch 5/500
13/13 [==============================] - 0s 2ms/step - loss: 5.1956 - mae: 5.1956
Epoch 6/500
13/13 [==============================] - 0s 1ms/step - loss: 5.3588 - mae: 5.3588
Epoch 7/500
13/13 [==============================] - 0s 1ms/step - loss: 5.5736 - mae: 5.5736
Epoch 8/500
13/13 [==============================] - 0s 2ms/step - loss: 5.4744 - mae: 5.4744
Epoch 9/500
13/13 [==============================] - 0s 2ms/step - loss: 5.7689 - mae: 5.7689
Epoch 10/500
13/13 [==============================] - 0s 2ms/step - loss: 5.1416 - mae: 5.1416
Epoch 11/500
13/13 [==============================] - 0s 

In [61]:
boston.evaluate(x_test,y_test)

4/4 [==============================] - 0s 814us/step - loss: 3.3091 - mae: 3.3091


[3.309069871902466, 3.309069871902466]